In [29]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
pd.options.plotting.backend = 'plotly'

## result

In [132]:
df_store = pd.read_pickle("data/df_daily.pkl")

score_ARIMA = pd.read_pickle("results/f8/ARIMA.pkl").set_index("store")
score_PROPHET = pd.read_pickle("results/f8/PROPHET.pkl").set_index("store")
score_SNAIVE = pd.read_pickle("results/f8/SNAIVE.pkl").set_index("store")

score_ALL = score_SNAIVE.join(score_ARIMA).join(score_PROPHET)
horizon = 30


In [129]:
score_ALL.describe().filter(items=['mean','std'], axis=0).filter(like='mae', axis=1)


,mae_SNAIVE,mae_ARIMA,mae_PROPHET
mean,8.191296,7.847007,8.129079
std,6.594733,5.777859,5.842856


In [130]:
score_ALL.describe().filter(items=['mean','std'], axis=0).filter(like='rmse', axis=1)


,rmse_SNAIVE,rmse_ARIMA,rmse_PROPHET
mean,11.167730,10.608467,10.686875
std,9.385727,7.997232,8.209956


In [ ]:
score_ALL.describe().filter(items=['mean','std'], axis=0).filter(like='mape', axis=1)


,mape_SNAIVE,mape_ARIMA,mape_PROPHET
mean,0.455632,0.452066,0.480375
std,0.085712,0.085296,0.120029


In [131]:
score_ALL.describe().filter(items=['mean','std'], axis=0).filter(like='mase', axis=1)


,mase_SNAIVE,mase_ARIMA,mase_PROPHET
mean,0.838204,1.068914,1.112441
std,0.312467,0.221264,0.229924


In [38]:
fig = score_ALL[['mae_SNAIVE', 'mae_ARIMA', 'mae_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAE')


In [41]:
fig = score_ALL[['rmse_SNAIVE', 'rmse_ARIMA', 'rmse_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='rmse')


In [39]:
fig = score_ALL[['mape_SNAIVE', 'mape_ARIMA', 'mape_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MAPE')


In [40]:
fig = score_ALL[['mase_SNAIVE', 'mase_ARIMA', 'mase_PROPHET']].plot()
fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='MASE')


In [114]:
sample_ls = ['store_307222', 'store_350018', 'store_566790']

y_true = [df_store.loc[df_store['store_id']==307222, 'sales'][-horizon:].values/1e6,
          df_store.loc[df_store['store_id']==350018, 'sales'][-horizon:].values/1e6,
          df_store.loc[df_store['store_id']==566790, 'sales'][-horizon:].values/1e6]
y_true=pd.DataFrame({'sales':y_true}, index=sample_ls)

sample_fc = score_ALL.filter(items=sample_ls, axis=0).filter(like='fc', axis=1).join(y_true)
sample_fc


,fc_SNAIVE,fc_ARIMA,fc_PROPHET,sales
store,,,,
store_307222,2021-01-02 32.368534 2021-01-03 34.00008...,"[37.029268763271205, 35.85993168857752, 20.326...",ds trend yhat_lower yhat_upp...,"[33.21181, 43.410811, 17.041805, 14.570605, 19..."
store_350018,2021-01-02 37.440195 2021-01-03 49.27070...,"[129.00611816929822, 140.6546586495469, 41.038...",ds trend yhat_lower yhat_upp...,"[131.5103, 113.6733, 31.6653, 40.9715, 24.8341..."
store_566790,2021-01-02 47.379806 2021-01-03 59.65101...,"[128.23982318681084, 116.41238961245561, 35.28...",ds trend yhat_lower yhat_upp...,"[94.345024, 60.149702, 14.22062, 17.5068, 18.0..."


In [122]:
fig = go.Figure()

store = sample_ls[0]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'dash': 'dash'},
                         ))
fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()


In [123]:
fig = go.Figure()

store = sample_ls[1]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'dash': 'dash'},
                         ))
fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()


In [124]:
fig = go.Figure()

store = sample_ls[2]
x = sample_fc.loc[store,'fc_SNAIVE'].index

fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'sales'], 
                         name='Actual sales',
                         line={'color': 'grey'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_SNAIVE'].values, 
                         name='SNAIVE',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_ARIMA'], 
                         name='SARIMAX',
                         line={'dash': 'dash'},
                         ))
fig.add_trace(go.Scatter(x=x, 
                         y=sample_fc.loc[store,'fc_PROPHET']['yhat'].values, 
                         name='PROPHET',
                         line={'dash': 'dash'},
                         ))
fig.update_layout(title='Sales Forecast ' + store)
fig.update_yaxes(title_text='Forecasts (million VND)')

fig.show()
